In [2]:
# Importação de pacotes
from selenium import webdriver
import time
import os
import base64
import import_ipynb
import shutil
import re
from math import ceil
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image, ImageChops

# Chromedriver configurations
chromeOptions = Options()
chromeOptions.add_argument("--start-maximized")
#driver = webdriver.Chrome('/home/pesquisador/Downloads/chromedriver')
driver = webdriver.Chrome('/Users/veronicastocco/NeuroMat/GoogleArts/chromedriver')
max_size = 12000

ModuleNotFoundError: No module named 'selenium'

In [2]:
# %load download_blobs.py
def set_up_the_painting (driver, URL_quadro):
    """
    Reduces Chromedriver's zoom to 25% (so more blobs can fit in)
    the window, increases GoogleArt's zoom to 100% (maximum resolution)
    and moves the window to the blob on the upper left.
    """
    driver.get(URL_quadro)
    time.sleep(5)
    driver.find_element_by_css_selector('.BfiV7c').click()
    time.sleep(3)
    for i in range(10):
        driver.find_element_by_css_selector('.XkWAb-eJuzjc .XkWAb-RCfa3e').click()
    time.sleep(3)
    url_x0_y0 = (driver.current_url).split("%2C%22z%22")
    driver.get('chrome://settings/')
    driver.execute_script('chrome.settingsPrivate.setDefaultZoom(.25);')
    driver.get("https://artsandculture.google.com/asset/"+base_url+"?ms=%7B%22x%22%3A" + str(0)+"%2C%22y%22%3A"+str(0)+"%2C%22z%22" + url_x0_y0[1])
    time.sleep(5)
    w, h, nx, ny = get_parameters()
    x = w / 2
    y = h / 2
    driver.get(URL_quadro)
    driver.get("https://artsandculture.google.com/asset/"+base_url+"?ms=%7B%22x%22%3A" + str(0)+"%2C%22y%22%3A"+str(0)+"%2C%22z%22%3A16%2C%22size%22%3A%7B%22width%22%3A"+str(w)+"%2C%22height%22%3A"+str(h)+"%7D%7D")

def get_parameters():
    """
    Extracts the following values from the GoogleArts url:

    w, h:   width and height of the image (depends on GoogleArts zoom, browser zoom and browser window size)
    nx, ny: number of times we'll need to move down and sideways
    """
    url = driver.current_url
    a = url.split("%22")
    print(a)

    w = float(a[10][3:-3])
    h = float(a[12][3:-6])
    
    x = w/2
    y = h/2

    nx = int((1-x)/(1.5*x)) + 1
    ny = int((1-y)/(1.5*y)) + 1
    return w, h, nx, ny

# função mágica que usa magia negra para salvar os blobs em imagens
def get_file_content_chrome(driver, uri):
    """
    Saves blob to base64.
    """
    result = driver.execute_async_script("""
    var uri = arguments[0];
    var callback = arguments[1];
    var toBase64 = function(buffer){for(var r,n=new Uint8Array(buffer),t=n.length,a=new Uint8Array(4*Math.ceil(t/3)),i=new Uint8Array(64),o=0,c=0;64>c;++c)i[c]="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/".charCodeAt(c);for(c=0;t-t%3>c;c+=3,o+=4)r=n[c]<<16|n[c+1]<<8|n[c+2],a[o]=i[r>>18],a[o+1]=i[r>>12&63],a[o+2]=i[r>>6&63],a[o+3]=i[63&r];return t%3===1?(r=n[t-1],a[o]=i[r>>2],a[o+1]=i[r<<4&63],a[o+2]=61,a[o+3]=61):t%3===2&&(r=(n[t-2]<<8)+n[t-1],a[o]=i[r>>10],a[o+1]=i[r>>4&63],a[o+2]=i[r<<2&63],a[o+3]=61),new TextDecoder("ascii").decode(a)};
    var xhr = new XMLHttpRequest();
    xhr.responseType = 'arraybuffer';
    xhr.onload = function(){ callback(toBase64(xhr.response)) };
    xhr.onerror = function(){ callback(xhr.status) };
    xhr.open('GET', uri);
    xhr.send();
    """, uri)
    if type(result) == int:
        raise Exception("Request failed with status %s" % result)
    return base64.b64decode(result)


def get_images(base_url):
    """
    Retrieves all blobs in the given url, saving them to a subfolder 'temp'.
    """
    w, h, nx, ny = get_parameters()
    x = w / 2
    y = h / 2
    nx = ceil(nx / 1.9)
    ny = ceil(ny / 1.9)
    base = "https://artsandculture.google.com/asset/"+base_url+"?ms=%7B%22x%22%3A"
    if (not os.path.isdir("temp")):
        os.mkdir("temp")
    os.chdir("temp")
    for xi in range(0 , nx + 2):
        for yi in range(ny + 2, ny + 3):
            print("loop x:{0}/{1} \t y:{2}/{3}".format(xi, nx, yi, ny))
            print(x+xi*x)
            print(y+yi*y)
            driver.get(base + str(x+xi*2*x)+"%2C%22y%22%3A"+str(y+yi*2*y)+"%2C%22z%22%3A16%2C%22size%22%3A%7B%22width%22%3A"+str(w)+"%2C%22height%22%3A"+str(h)+"%7D%7D")
            time.sleep(20)
            blobs = driver.find_elements_by_tag_name('img')
            j = 0

            for blob in blobs:
                # the first two blobs are miniatures of the painting
                if j > 2:
                    time.sleep(0.2)
                    # Get number of rows and columns
                    style = blob.get_attribute('style')
                    style_end_index = style.find(');')
                    # -4 removes "z" translation
                    style = style[:style_end_index - 4]
                    style = style.replace('transform: translate3d(', '')
                    style= style.split(",")

                    try:
                        filename = '{:07d} - {:07d}.jpg'.format(int(style[0][:-2]), int(style[1][:-2]))
                        print(filename)
                        if os.path.exists(filename):
                            pass

                        else:
                            # Save blob to file
                            image = (get_file_content_chrome(driver, blob.get_attribute('src')))
                            with open(filename, 'wb') as f:
                                f.write(image)
                    except:
                        #invalid file name
                        print(filename)
                        pass
                j+=1


In [3]:
# %load treat_images.py
def size_and_xy (blobs):
    """
    Returns the number of rows (y) and columns (x) of blobs in the 
    final image blased on the coordinates of the last blob,
    as well as the size of the blob (in pixels).
    """
    size = int(blobs[1].split('-')[1][1:-4]) #gets the size of the blob 
    a = blobs[-1].split('-')
    x = int(a[0]) / size
    y = int(a[1][1:-4]) / size
    return (int(x) + 1, int(y) + 1, size)

def get_x (file_name):
    """
    Returns the column where file_name should be placed
    """
    x = file_name.split('-')[0]
    return (int(x))

def get_y (file_name):
    """
    Returns the row where file_name should be placed
    """
    y = file_name.split('-')[0]
    y = file_name[1:-4]
    print(y)
    return (int(y))

def check_unnecessary_images (blobs, size):
    """
    Deletes unnecesary images that may have been downloaded.
    """
    if (len(blobs) > x * y):

        deleted = 0
        try:
            os.mkdir("trash")
        except:
            pass
            
        for b in blobs:
            try:
                title = int(b.split("-")[0])
                if (title % size != 0):
                    print(b)
                    os.rename(b, "trash/" + b)
                    deleted +=1
                    blobs.remove(b)

            except:
                print(b)
                os.rename(b, "trash/" + b)
                deleted +=1
                blobs.remove(b)
        print("{0} extra file(s) was/were removed.\n".format(deleted))
    return blobs
        

def define_crop(size, x, y, max_size):
    """
    Returns the maximum size of the full resolution slices, 
    based on the size of the blob, the number of rows (y) and
    columns (x) and max_size defined by the user.
    """
    max_x = ceil(x * size / max_size)
    max_y = ceil(y * size / max_size)
    return (max_x, max_y)
    
def define_resize(size, x, y, max_size):
    """
    Returns the size the blobs must have to generate an image
    with the complete artwork that is smaller than max_size.
    """
    n = max(x, y)
    resize = size
    while (n * resize > max_size):
        resize = resize / 2
    return int(resize);

def smaller_blobs(resize, blobs):
    """
    Resizes all blobs to the size defined in 'resize'.
    """
    os.mkdir("smaller")
    for blob in blobs:
        image = Image.open(blob)
        image = image.resize((resize, resize), Image.LANCZOS)
        image.save('smaller/{0}'.format(blob), dpi=[300,300])
    os.chdir("smaller")
    if(verbose):
        print("Created all smaller blobs.\n")


def get_sections(blobs, x, y, cx, cy, x_i, y_i):
    base_x = cx * x_i
    base_y = cy * y_i
    imgs = []
    debug = []
    for i in range(cx):
    # column is outside of the section
        for j in range (cy):
            # line is outside of the section
            n = base_x + base_y + i * y + j
            imgs.append(blobs[n])
            debug.append(n)
    if y_i is (max_y - 1):
        y_section = cy
    else:
        y_section = y % cy
        print(" a")
    os.mkdir("cols")
    print(debug)
    print(len(imgs))
    print(x_section)
    print(y_section)
    cols = concatenate_cols(imgs, x_section + 1, y_section)
    #print(len(cols))
    title = "{0} - {1} .jpg".format(x_i, y_i)
    merge_image(cols, title)
    shutil.rmtree('cols')



def split(l, n):
    """
    Splits the list l in n slices, and returns it as a generator.
    """
    for i in range(0, len(l), n):
        yield l[i:i + n]

def concatenate_partial_cols(blobs, x, y, max_y, cy):
    """
    Concatenates the columns that form each of the slices with
    maximum resolution.
    """
    cols = []
    os.mkdir("cols")
    for col in range (x):
        imgs = []
        for j in range (col * y, col * y + y):
            imgs.append(blobs[j])
        images = list(map(Image.open, imgs))

        count = 0
        for l in split(images, cy):
            y_offset = 0
            widths, heights = zip(*(i.size for i in l))
            t_width = max(widths)
            t_height = sum(heights)
            new_im = Image.new('RGB', (t_width, t_height))
            for im in l:
              new_im.paste(im, (0, y_offset))
              y_offset += im.size[1]

            new_im.save('cols/col_{0:03d}_{1:03d}.jpg'.format(count, col), dpi=[300,300])
            cols.append('cols/col_{0:03d}_{1:03d}.jpg'.format(count, col))
            count += 1
    if (verbose):
        print("Created all partial columns.\n")
    return cols


def split_helper(l, cx, x):
    """
    Splits all columns into the slices of the maximum resolution
    image they should be in, and returns it as a list.
    """
    new = []
    check = cx
    val = cx
    while(len(l) > 0):
        new.append(l[:val])
        l = l[val:]
        check += cx
        if (check > x):
            val = x - val
            check = cx
    return new
        
def merge_partial_images(cols, name, max_x, cx, cy):
    """
    Merges the columns that will form the final slices with
    full resolution of the artwork.
    """
    cols = sorted(cols)
    count_x = 0
    count_y = 0
    for sub_cols in split_helper(cols, cx, x):
        images = list(map(Image.open, sub_cols))
        widths, heights = zip(*(i.size for i in images))
        im_width = sum(widths)
        im_height = max(heights)

        new_im = Image.new('RGB', (im_width, im_height))

        x_offset = 0
        for c in images:
            new_im.paste(c, (x_offset, 0))
            x_offset += c.size[0]
        file_at_root = root + "/Done/" + '{0} ({1} - {2}).jpg'.format(name, count_x, count_y)
        new_im.save(file_at_root, dpi=[300,300])
        count_x += 1
                        
        if count_x > (max_x - 1):
            count_x = 0
            count_y += 1
    if (verbose):
        print("Merged all partial columns.\n")

def concatenate_cols(blobs, x, y):
    """
    Concatenates all blobs into columns. Used to form the 
    image wih resized blobs.
    """
    cols = []
    os.mkdir("cols")
    for col in range (x):
        imgs = []
        for j in range (col * y, col * y + y):
            imgs.append(blobs[j])
        images = list(map(Image.open, imgs))
        widths, heights = zip(*(i.size for i in images))
        t_width = max(widths)
        t_height = sum(heights)
        
        new_im = Image.new('RGB', (t_width, t_height))

        y_offset = 0

        for im in images:
          new_im.paste(im, (0, y_offset))
          y_offset += im.size[1]

        new_im.save('cols/col_{0}.jpg'.format(col), dpi=[300,300])
        cols.append('cols/col_{0}.jpg'.format(col))
    if (verbose):
        print("Merged all columns.\n")
    return cols

def merge_image(cols, name):
    """
    Merges the columns of resized blobs into one full image
    of the artwork.
    """
    images = list(map(Image.open, cols))
    widths, heights = zip(*(i.size for i in images))
    im_width = sum(widths)
    im_height = max(heights)

    new_im = Image.new('RGB', (im_width, im_height))

    x_offset = 0
    for c in images:
        new_im.paste(c, (x_offset, 0))
        x_offset += c.size[0]

    file_at_root = root + "/Done/" + name + " (resized).jpg"
    new_im.save(file_at_root, dpi=[300,300])
    if (verbose):
        print("Created {0}.\n".format(name))

def trim(nome):
    im = Image.open(nome)
    x,y = im.size
    x -= 1
    y -= 1
    bg = Image.new(im.mode, im.size, im.getpixel((x,y)))
    diff = ImageChops.difference(im, bg)
    diff = ImageChops.add(diff, diff, 2.0, 0)
    box = diff.getbbox()
    if box:
        imagem = im.crop(box)
        imagem.save(nome)

        
def find_borders (blobs, x, y):
    # last row
    for i in range(x):
        trim(blobs[i * y - 1])
    # last column
    for i in range(x * y - y, x * y):
        trim(blobs[i])
    if (verbose):
        print("Trimmed all black areas.")

def get_blobs (location=""):
    path = os.getcwd() + "/" + location
    b = os.listdir(path)
    blobs = [x for x in b if re.match("[0-9-]+", x)]

    return sorted(blobs)

In [4]:
# Painting URL
URL_quadro = "https://artsandculture.google.com/asset/reminisc%C3%AAncia-do-morro-do-castelo/yAEpUz3BkUYdXg"
base_url = URL_quadro.split('asset/')[1].split('?ms')[0]
name = URL_quadro.split('/')[4]

# Blobs class
class_code = "XkWAb-LmsqOc"

#root directory
root = os.getcwd()


In [5]:
set_up_the_painting (driver, URL_quadro)
title_of_work = driver.title.split("-")[0]

['https://artsandculture.google.com/asset/reminisc%C3%AAncia-do-morro-do-castelo/yAEpUz3BkUYdXg?ms=%7B', 'x', '%3A0.055247330877896406%2C', 'y', '%3A0.053475935828877%2C', 'z', '%3A15%2C', 'size', '%3A%7B', 'width', '%3A0.11049466175579281%2C', 'height', '%3A0.10695187165775401%7D%7D']


In [6]:
get_images (base_url)

['https://artsandculture.google.com/asset/reminisc%C3%AAncia-do-morro-do-castelo/yAEpUz3BkUYdXg?ms=%7B', 'x', '%3A0%2C', 'y', '%3A0%2C', 'z', '%3A16%2C', 'size', '%3A%7B', 'width', '%3A0.11049466175579281%2C', 'height', '%3A0.10695187165775401%7D%7D']
loop x:0/7 	 y:9/7
0.055247330877896406
0.5347593582887701
0000000 - 0023552.jpg
0000000 - 0024064.jpg
0000000 - 0024576.jpg
0000000 - 0025088.jpg
0000000 - 0025600.jpg
0000000 - 0026112.jpg
0000512 - 0023552.jpg
0000512 - 0024064.jpg
0000512 - 0024576.jpg
0000512 - 0025088.jpg
0000512 - 0025600.jpg
0000512 - 0026112.jpg
0001024 - 0023552.jpg
0001024 - 0024064.jpg
0001024 - 0024576.jpg
0001024 - 0025088.jpg
0001024 - 0025600.jpg
0001024 - 0026112.jpg
0001536 - 0023552.jpg
0001536 - 0024064.jpg
0001536 - 0024576.jpg
0001536 - 0025088.jpg
0001536 - 0025600.jpg
0001536 - 0026112.jpg
0002048 - 0023552.jpg
0002048 - 0024064.jpg
0002048 - 0024576.jpg
0002048 - 0025088.jpg
0002048 - 0025600.jpg
0002048 - 0026112.jpg
0002560 - 0023552.jpg
0002560

0026112 - 0026112.jpg
0026624 - 0023552.jpg
0026624 - 0024064.jpg
0026624 - 0024576.jpg
0026624 - 0025088.jpg
0026624 - 0025600.jpg
0026624 - 0026112.jpg
0027136 - 0023552.jpg
0027136 - 0024064.jpg
0027136 - 0024576.jpg
0027136 - 0025088.jpg
0027136 - 0025600.jpg
0027136 - 0026112.jpg
0027648 - 0023552.jpg
0027648 - 0024064.jpg
0027648 - 0024576.jpg
0027648 - 0025088.jpg
0027648 - 0025600.jpg
0027648 - 0026112.jpg
0028160 - 0023552.jpg
0028160 - 0024064.jpg
0028160 - 0024576.jpg
0028160 - 0025088.jpg
0028160 - 0025600.jpg
0028160 - 0026112.jpg
0028160 - 0026112.jpg
loop x:6/7 	 y:9/7
0.38673131614527484
0.5347593582887701
0028160 - 0023552.jpg
0028160 - 0024064.jpg
0028160 - 0024576.jpg
0028160 - 0025088.jpg
0028160 - 0025600.jpg
0028160 - 0026112.jpg
0028672 - 0023552.jpg
0028672 - 0024064.jpg
0028672 - 0024576.jpg
0028672 - 0025088.jpg
0028672 - 0025600.jpg
0028672 - 0026112.jpg
0029184 - 0023552.jpg
0029184 - 0024064.jpg
0029184 - 0024576.jpg
0029184 - 0025088.jpg
0029184 - 0025600.

In [7]:
verbose = True
#os.chdir("temp")

In [9]:

blobs = get_blobs()
x, y, size = size_and_xy(blobs)
blobs = check_unnecessary_images(blobs, size)
find_borders(blobs, x, y)
resize = define_resize(size, x, y, max_size)
max_x, max_y = define_crop(size, x, y, max_size)
cx = ceil(x / max_x)
cy = ceil(y / max_y)
partial_cols = concatenate_partial_cols(blobs, x, y, max_y, cy)


Trimmed all black areas.
Created all partial columns.



In [ ]:
merge_partial_images(partial_cols, title_of_work, max_x, cx, cy)
smaller_blobs(resize, blobs)
small_blobs = blobs
small_cols = concatenate_cols(small_blobs, x, y)
merge_image(small_cols, title_of_work)


Merged all partial columns.

Created all smaller blobs.

Merged all columns.

